In [ ]:
import numpy as np
import re
from timeit import default_timer as timer

In [ ]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read().split('\n')

In [ ]:
class Sensor():
    def __init__(self, xs, ys, xb, yb):
        self.x = xs
        self.y = ys
        self.x_beacon = xb
        self.y_beacon = yb
        
        self.d_closest_beacon = self.distance_to(xb, yb)
    
    def distance_to(self, x, y):
        return abs(y - self.y) + abs(x - self.x)
    
    def points_covered_at_row(self, y):

        d_to_row = self.distance_to(self.x, y)
        if d_to_row > self.d_closest_beacon:
            return set()
        else:
            points = [(self.x, y)]
            
            for dx in range(1, self.d_closest_beacon - d_to_row + 1):
                points += [(self.x - dx, y), (self.x + dx, y)]
            
            return set(points)

    
    def get_edge_points(self, minimum, maximum):
        d = self.d_closest_beacon
        points = []
        for i in range(d + 2):
            points += [ (0 + i, d+1 - i), 
                        (0 + i, -(d+1) + i),
                        (0 - i, d+1 - i), 
                        (0 - i, -(d+1) + i)
                        ]
        points = list(set(points))

        for p in points:
            new_x = self.x + p[1]
            new_y = self.y + p[0]
            if (new_x >= minimum) and (new_x <= maximum) and (new_y >= minimum) and (new_y <= maximum):
                yield (new_x, new_y)
        
    def is_seen(self, point):
        return self.distance_to(point[0], point[1]) <= self.d_closest_beacon
    
    def get_corner_points(self):
        return [(self.x, self.y + self.d_closest_beacon),
                (self.x, self.y - self.d_closest_beacon),
                (self.x + self.d_closest_beacon, self.y),
                (self.x - self.d_closest_beacon, self.y)]

# Part 1

In [80]:
x = get_input('input')

In [81]:
sensors = []
for s in x:
    xs,ys, xb,yb = [int(c) for c in re.findall('=([-+]?[0-9]*)',s)]
    sensors.append(Sensor(xs,ys,xb,yb))

In [113]:
points_covered = set()
beacons = set()
row = 2000000
for sensor in sensors:
    beacons = beacons.union( set( [(sensor.x_beacon, sensor.y_beacon)] ) )
    points_covered = points_covered.union( sensor.points_covered_at_row(row) )

In [110]:
len(points_covered.difference(beacons))

4879972

# Part 2

In [87]:
x = get_input('input')

In [88]:
sensors = []
for s in x:
    xs,ys, xb,yb = [int(c) for c in re.findall('=([-+]?[0-9]*)',s)]
    sensors.append(Sensor(xs,ys,xb,yb))

In [89]:
t0 = timer()
for sensor in sensors:
    
    for p in sensor.get_edge_points(0, 4000000):
        seen = False
        for sensor in sensors:
            if sensor.is_seen(p):
                seen = True
                break

        if not seen:
            target = p
print(f'{(timer()-t0)/60:.2f} min')

5.01 min


In [90]:
print(target[0]*4000000 + target[1])

12525726647448


In [91]:
target

(3131431, 2647448)

# Part 2 with line intersects

In [98]:
x = get_input('test')

In [99]:
sensors = []
for s in x:
    xs,ys, xb,yb = [int(c) for c in re.findall('=([-+]?[0-9]*)',s)]
    sensors.append(Sensor(xs,ys,xb,yb))

In [100]:
def add(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))

def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return int(x), int(y)
    else:
        return False

In [93]:
tr = (3131431, 2647448)

In [106]:
def allowed(p, mini, maxi):
    return ((max(p) <= maxi) & (min(p) >= mini))

lines = []
for sensor in sensors:
    corners = sensor.get_corner_points()
    for c1,c2 in zip(corners, corners[1:]+[corners[0]]):
        lines.append(line(c1, c2))

overlaps = []
for L1, L2 in itertools.combinations(lines, 2):
    if (pt := intersection(L1, L2)):
        overlaps.append(pt)

for pt in overlaps:
    for d in [(0,1), (0,-1), (1,0), (-1,0)]:
        nei = add(pt, d)
        # 4000000
        if allowed(nei, 0, 20):
            seen = False
            for sensor in sensors:
                if sensor.is_seen(nei):
                    seen = True
                    break

            if not seen:
                target = nei
                
print(target[0]*4000000 + target[1])

56000011


In [107]:
target

(14, 11)

In [102]:
def manhattan(p1, p2):
    return abs(p2[1] - p1[1]) + abs(p2[0] - p1[0])

In [110]:
# for ol in overlaps:
#     print(manhattan(ol, target))

In [78]:
l1 = line((0,1), (0,2))

In [80]:
l2 = line((0, 5), (0, 6))

In [82]:
intersection(l1, l2)

False